In [97]:
import json
# đọc các file ocr và annotation 
with open('D:/MMSD/data/train/output_data2.json', 'r', encoding='utf-8') as f:
    ocr_lines = json.load(f)
with open('D:/MMSD/data/train/vimmsd-train.json', 'r', encoding='utf-8') as f:
    annotation_lines = json.load(f)

In [ ]:
from datasets import load_dataset
ds = load_dataset("tsdocode/vietnamese-dictionary")

d:\MMSD\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\MMSD\venv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\datasets--tsdocode--vietnamese-dictionary. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https

In [51]:
vietnamese_vocab=ds['train']['word']

In [ ]:
import re
import unicodedata
import emoji
from difflib import SequenceMatcher
from spellchecker import SpellChecker



def preprocess_vietnamese_text(column,stopwords=None):
    for row in column:
        try:
            row = str(row).lower()  # Chuyển tất cả thành chữ thường

            # Chuẩn hóa Unicode
            row = unicodedata.normalize('NFC', row)

         
            row = re.sub(r"[#%^*+={}\\|<>[\]\"']", ' ', row)
            row = re.sub(r"[\.\,\!\?\:\;\-\_]", ' ', row)

       
            row = re.sub(r"(\\t|\\r|\\n)", ' ', row)

            # Loại bỏ ký tự đặc biệt liên tiếp
            row = re.sub(r"(__+)", ' ', row)
            row = re.sub(r"(--+)", ' ', row)
            row = re.sub(r"(~~+)", ' ', row)
            row = re.sub(r"(\+\++)", ' ', row)
            row = re.sub(r"(\.\.+)", ' ', row)

            # Loại bỏ các ký tự đặc biệt khác
            row = re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]", ' ', row)

            # Thay thế các mẫu ký tự đặc biệt
            row = re.sub(r"(mailto:)", ' ', row)
            row = re.sub(r"(\\x9\d)", ' ', row)
            row = re.sub(r"([iI][nN][cC]\d+)", 'INC_NUM', row)
            row = re.sub(r"([cC][mM]\d+)|([cC][hH][gG]\d+)", 'CM_NUM', row)

            # Thay thế emoji bằng <emoji>
            row = emoji.replace_emoji(row, replace="<emoji>")

            row = re.sub(r'[+\-*/=<>]', "", row)

            # Loại bỏ khoảng trắng thừa và các ký tự không cần thiết
            row = re.sub(r"(\.\s+)", ' ', row)
            row = re.sub(r"(\-\s+)", ' ', row)
            row = re.sub(r"(\:\s+)", ' ', row)
            row = re.sub(r"(\s+.\s+)", ' ', row)  # Loại bỏ ký tự đơn giữa các khoảng trắng

            # Thay thế URL thành domain name
            try:
                url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', row)
                repl_url = url.group(3)
                row = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', repl_url, row)
            except:
                pass
            
           
            row = re.sub(r'\b(\w+)( \1\b)+', r'\1', row)  

             # Sửa chính tả hoặc loại bỏ từ sai
    
            corrected_row = []
            for word in row.split():
                if word.isdigit():  # Kiểm tra nếu từ là số
                    corrected_row.append(word)  # Giữ nguyên số
                else:
                    correction = correct_spelling(word)
                    if correction and is_correctable(word, correction):
                        corrected_row.append(correction)  # Thêm từ đã sửa
        
            # Ghép lại thành chuỗi
            row = ' '.join(corrected_row)
            row = re.sub(r"(\s+)", ' ', row).strip()   # Loại bỏ nhiều khoảng trắng liên tiếp
            
        except:
            pass
        yield row

def process_json_ocr(data,stopwords):
    for key, value in data.items():
        ocr_text = value['ocr']  # Lấy giá trị ocr
        # Xử lý văn bản trong trường 'ocr'
        processed_text = next(preprocess_vietnamese_text([ocr_text],stopwords=stopwords))
        value['ocr'] = processed_text  # Cập nhật lại giá trị 'ocr'

    return data

def process_json_annotation(data,stopwords):
    for key, value in data.items():
        # Lấy giá trị 'text' trong trường 'annotation'
        annotation_text = value['caption']
        # Xử lý văn bản trong trường 'text'
        processed_text = next(preprocess_vietnamese_text([annotation_text],stopwords=stopwords))
        value['caption'] = processed_text  # Cập nhật lại giá trị 'text'

    return data
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        stopwords = set(line.strip() for line in f)  # Tạo một set các từ stopword
    return stopwords


def correct_spelling(word):
    # Tạo đối tượng SpellChecker với từ điển tiếng Việt
    spell = SpellChecker(language=None)  # Không dùng ngôn ngữ mặc định
    if word in vietnamese_vocab:  # Từ hợp lệ
        return word
    correction = spell.correction(word)  # Sửa từ
    if correction is not None and correction in vietnamese_vocab:
        return correction
    return None  # Loại bỏ từ không thể sửa



def is_correctable(word, correction, threshold=2):
    if correction is None:
        return False
    # Tính khoảng cách chỉnh sửa
    edit_distance = SequenceMatcher(None, word, correction).ratio()
    return edit_distance > (1 - threshold / max(len(word), len(correction)))

In [98]:
stopwords = load_stopwords('D:/MMSD/vietnamese-stopwords.txt')
ocr_lines = process_json_ocr(ocr_lines,stopwords)
ocr_lines

{'0': {'image': '8ae451edcd8ebf697f8763ece249115813149c55733bf8306b971210d48b15d3.jpg',
  'ocr': ''},
 '1': {'image': '35370ffd6c791d6f8c4ab3dd4363ed468fab41e4824ee9984169f58be672a0b7.jpg',
  'ocr': 'chốn bàn thờ đột bị đổ rơi nát tanh bành tượng luôn có sao 8 giờ trả lời 075 người ấy do bắt vít không chặt ấy giờ trả lời 01 353 lông xuống'},
 '2': {'image': '316fdd1477725b9fb1a55015ac06b68b92b50bd4303e08590dadedace629ae61.jpg',
  'ocr': ''},
 '3': {'image': '8a0f34e0e30e4e5cfb306933c1d25fa801a5da78646b59ca90a3a3c081cdb16c.jpg',
  'ocr': 'nhiều người nam đi làm chỉ mong đến đúng chiều là về đã vậy lại rất hay xin nghỉ giỗ ông nội cũng nghỉ sinh bà ngoại cũng nghỉ'},
 '4': {'image': 'e517a5e95d1065886a7c815e82fe254381d4f9f4b244d48179d2a4152d3146da.jpg',
  'ocr': ''},
 '5': {'image': '1a972a37d1cd6990baa5c4ec28c053608b71482e30b134faa66b345d87a3a1ff.jpg',
  'ocr': 'trà lời bình luận của lúc về kia đèo váy hoa tay đeo đồng hồ thì đi mình kia nhẫn nha còn tay thì có nhẫn ngón áp út nha tay'}

In [ ]:
annotation_lines = process_json_annotation(annotation_lines,stopwords)
annotation_lines
with open('D:/MMSD/data/train/ocr_processed.json', 'w', encoding='utf-8') as f:
    json.dump(ocr_lines, f, ensure_ascii=False, indent=4)
with open('D:/MMSD/data/train/vimmsd-train_processed.json', 'w', encoding='utf-8') as f:
    json.dump(annotation_lines, f, ensure_ascii=False, indent=4)